In [ ]:
## In this we basically applied feature engineering techniques like one hot encoding and label encoding to clean the data for ml models
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [68]:
## Load dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head() #to view the first few rows

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [69]:
## WE have to predict the 'Exited' column that indicates whether a customer has churned (1) or not (0)
## REMOVE unnecessary columns like 'RowNumber', 'CustomerId', 'Surname'
data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [70]:
## Encoding categorical variables like 'Geography' and 'Gender'  to numerical values by 1 and 0 (feture engineering)
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data   ## labelencoding is one type of technique to convert categorical data into numerical data  we can also use one hot encoding technique etc.

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [71]:
## One Hot Encoding for 'Geography' column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder= onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder 


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [72]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [73]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))   ## .toarray() is used to convert sparse matrix to dense matrix IF NOT USE IT WILL SHOW ERROR
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [74]:
## Combine one hot encoded columns with original dataset and drop the original 'Geography' column
data = pd.concat([data, geo_encoded_df], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,France,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,France,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [75]:
# Now drop the original 'Geography' column
data = data.drop(columns=['Geography'], axis=1) 
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [76]:
## Save the latest encoder and scaler changes using pickle 
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)



## What is the use of `pickle.dump()` in `wb` mode?

### Meaning of `wb`
- **w** → write (file likhne ke liye)
- **b** → binary (machine-readable format)

👉 Pickle binary data likhta hai  
👉 Isliye `wb` mode **mandatory** hota hai  

Agar sirf `w` (text mode) use kiya:
- File corrupt ho sakti hai
- Ya error aa sakta hai

---

### What does `pickle.dump()` do?
- Python object ko **byte stream** mein convert karta hai
- Us object ko file ke andar store karta hai

---

### Examples of objects saved using pickle
- Trained ML model  
- LabelEncoder / OneHotEncoder  
- Scaler (StandardScaler, MinMaxScaler)  
- Any Python object

---

### Why binary mode is required?
ML models ke andar hota hai:
- Weights  
- Arrays  
- Metadata  

Ye **text nahi hote**, raw binary bytes hote hain.  
Text mode (`w`) inhe damage kar deta hai, isliye `wb` use hota hai.

---

### Example Code
```python
import pickle

with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [ ]:
## Divide the dataset into independent and dependent features
X = data.drop('Exited', axis=1)  # Independent features
y = data['Exited']               # Dependent feature

## Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Feature Scaling using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  #fit_transform only applied on training data
X_test = scaler.transform(X_test) #transform only applied on testing data(already-learned transformation from training data)



In [60]:
y_test

6252    0
4684    0
1731    0
4742    0
4521    0
       ..
6412    1
8285    0
7853    1
1095    1
6929    1
Name: Exited, Length: 2000, dtype: int64

In [78]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

ANN Implementation

In [81]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import datetime

In [86]:
## Now time to build our ANN model
model = Sequential([
    Dense(64 , activation = 'relu' , input_shape =(X_train.shape[1],)), ##HL1 with 64 neuron and X_train is send as input in it
    Dense(32 , activation='relu'), #no need of again giving input as it is sequential and it will be connected to each other hl1 to hl2
    Dense(1,activation='sigmoid') ##output layer
])
##List is used above

In [ ]:
model.summary()
## Param stated below is basically combination of bias and weights

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [95]:
## To do forward and backward propogation we need to compile the model 
model.compile(optimizer="adam" , loss = "binary_crossentropy" , metrics=['accuracy'] ) ## if it is multiclass then it is sparse_crossentropy.



In [ ]:
## WE can set our own learning rate as well then no need of above code 
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate= 0.01) ##our custom learning rate set.
loss =tensorflow.keras.losses.BinaryCrossentropy()

<LossFunctionWrapper(<function binary_crossentropy at 0x000001C40371A0C0>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [ ]:
## After using above code we can write this also 
model.compile(optimizer= opt , loss = "binary_crossentropy" , metrics=['accuracy'] ) 
## we can also use loss = loss  instead of "binary_crossentropy"
## Can use opt insted of adam

In [101]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping 

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")  ## when we train our model all logs will be saved in this folder
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)  ## we can use matabolib insted of histogram freq=1


In [106]:
## Set Early Stopping to prevent  overfitting agar loss decrease nhi ho rha to training band ho sake
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

Epochs = how many times the model sees the entire training dataset and updates its weights.

Fewer epochs → underfitting

Too many epochs → overfitting

With early stopping, epochs is just the maximum limit

Example:
epochs=100 means “train up to 100 full passes, but stop early if validation(loss) stops improving(decreasing).”

In [107]:
### Train the model
history = model.fit(X_train, y_train , validation_data=(X_test, y_test), epochs=100, callbacks=[tensorflow_callback, early_stopping])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8684 - loss: 0.3220 - val_accuracy: 0.8645 - val_loss: 0.3372
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8706 - loss: 0.3198 - val_accuracy: 0.8595 - val_loss: 0.3382
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8676 - loss: 0.3185 - val_accuracy: 0.8620 - val_loss: 0.3402
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8687 - loss: 0.3180 - val_accuracy: 0.8585 - val_loss: 0.3461
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8685 - loss: 0.3141 - val_accuracy: 0.8635 - val_loss: 0.3403
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8649 - loss: 0.3129 - val_accuracy: 0.8605 - val_loss: 0.3392
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8710 - loss: 0.3104 - val_accuracy: 0.8630 - val_loss: 0.3433
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8701 - loss: 0.3089 - val_a

In [108]:
model.save('model.h5') ### Save the trained model

In [111]:
### Load the trained model
from tensorflow.keras.models import load_model
model = load_model('model.h5')
### Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8645 - loss: 0.3372 
Test Loss: 0.33722081780433655
Test Accuracy: 0.8644999861717224


In [114]:
### Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [115]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 5916), started 0:01:12 ago. (Use '!kill 5916' to kill it.)